In [1]:
%matplotlib inline
import torch
from sebm.sgld import SGLD_sampler
from sebm.models import EBM
dataset =  'svhn' # 'svhn' # 'cifar10' # 'mnist' #  'flowers102' #
if dataset == 'mnist' or dataset =='fashionmnist':
    input_channels, im_height, im_width = 1, 28, 28
else:
    input_channels, im_height, im_width = 3, 32, 32
device = torch.device('cuda:1')
arch =  'simplenet'
seed = 123
# lr, reg_alpha = 5e-5, 5e-3
lr, reg_alpha = 1e-4, 1e-1
# lr, reg_alpha = 5e-5, 5e-3
optimize_priors = False
if dataset == 'cifar10' or dataset == 'svhn':
    channels, kernels, strides, paddings =[64,128,256,512], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    hidden_dim, latent_dim, activation = [1024,128], 1, 'Swish'
elif dataset == 'mnist' or dataset == 'fashionmnist':
    channels, kernels, strides, paddings =[64,64,32,32], [3,4,4,4], [1,2,2,2], [1,1,1,1]
    hidden_dim, latent_dim, activation = [128,128], 1, 'Swish'
else:
    raise NotImplementError
data_noise_std = 3e-2
sgld_noise_std, sgld_lr, sgld_num_steps = 7.5e-3, 2.0, 60
buffer_init, buffer_dup_allowed = True, True
data_dir = '../../../sebm_data/'
load_version = 'igebm-d=%s-seed=%d-lr=%s-zd=%d-d_ns=%s-sgld-ns=%s-lr=%s-steps=%s-reg=%s-act=%s-arch=%s' % (dataset, seed, lr, latent_dim, data_noise_std, sgld_noise_std, sgld_lr, sgld_num_steps,  reg_alpha, activation, arch)

ebm = EBM(arch=arch,
                im_height=im_height, 
                im_width=im_width, 
                input_channels=input_channels, 
                channels=channels, 
                kernels=kernels, 
                strides=strides, 
                paddings=paddings, 
                hidden_dim=hidden_dim,
                latent_dim=latent_dim,
                activation=activation)
ebm = ebm.cuda().to(device)
print('Loading trained weights..')
# ebm.load_state_dict(torch.load('../weights/final/cp-%s' % load_version)['model_state_dict'])

Loading trained weights..


In [2]:
ebm

EBM(
  (ebm_net): SimpleNet(
    (cnn_block): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): Swish()
      (2): Conv2d(64, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (3): Swish()
      (4): Conv2d(128, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (5): Swish()
      (6): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1))
      (7): Swish()
    )
    (flatten): Flatten()
    (mlp_block): Sequential(
      (0): Linear(in_features=8192, out_features=1024, bias=True)
      (1): Swish()
      (2): Linear(in_features=1024, out_features=128, bias=True)
      (3): Swish()
      (4): Linear(in_features=128, out_features=1, bias=True)
    )
  )
)

In [2]:
from sebm.eval import *
evaluator = Evaluator_EBM(ebm, device, dataset, data_dir, data_noise_std=1e-2)

In [14]:
# fewshots(model_name='ebm', evaluator=evaluator)

In [ ]:
similarity_z_space(evaluator, 
                       train_batch_size=2000, 
                       test_batch_size=2000,
                       model_name='igebm')

In [ ]:
%debug

In [ ]:
# paired = torch.cat((ys_test.unsqueeze(-1), pred_ys_test), -1)
# torch.save(paired, 'confusion_matrix_labels_igebm_z_%s.pt' % dataset)

In [ ]:
# scores = evaluator.oodauc(dataset_ood='texture', score='gradient')
# scores

In [ ]:
images_ebm = evaluator.uncond_sampling(batch_size=100, 
                                          sgld_steps=1000,
                                          sgld_lr=2.0,
                                          sgld_noise_std=1e-3,
                                          grad_clipping=False,
                                          init_samples=None,
                                          logging_interval=None)
evaluator.plot_final_samples(images_ebm, fs=5)

In [ ]:
evaluator.plot_oods(dataset_ood='cifar10', score='gradient',train=False, fs=4, save=False)